In [5]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Loading

In [29]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'

donor_type ='Deceased'
# donor_type = 'All'
# antibody_epitope = 
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']
one_hot_encoding = ['TypeCadaveric_NOTR', 'Donor_Type']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'TypeCadaveric_NOTR', 
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'TypeCadaveric_NOTR', 'IL2rMoAb_T0')
    .pipe(features_from_antibody_epitopes)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(feature_scaler, num_col, scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)    

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.017022s
- selceted cohort --> Donor Type: Deceased, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (3235, 23) | Computation Time: 0.007003s
Step: integer_encoder | Shape: (3235, 13) | Computation Time: 0.002289s
Step: set_time_event_label | Shape: (3235, 16) | Computation Time: 0.002048s
Step: censoring_deaths | Shape: (3235, 16) | Computation Time: 0.001127s
Step: setting_prediction_horizon | Shape: (3235, 16) | Computation Time: 0.004504s


# Checking the Cox Proportional Hazard Assumptions

https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html?highlight=cox%20assumptions

The proportional hazard assumption is that all individuals have the same hazard function, but a unique scaling factor infront. So the shape of the hazard function is the same for all individuals, and only a scalar multiple changes per individual.

## Finding Inverse Probability Wieghts (IPW)

In [50]:
from lifelines import CoxPHFitter

df_new = df.copy(deep=True)
df_new['Class_I_II'] = df_new.Class_I * df_new.Class_II
poly_col = ['RecipientAge_NOTR', 'DonorAge_NOTR']
# for col in poly_col:
#     df_new[col +'^2'] = df_new[col] * df_new[col]
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df_new, RELEVANT_DESA_BAD)
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
# treatments = ['Relevant_DESA_Bad', 'Relevant_DESA_Good']
df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)

cph = CoxPHFitter()
# df_new = df_weight.drop(
#     [
#         '#DESA', 'Donor_Type', 'TypeCadaveric_NOTR', 
#         'DESA',
#         'Class_I',
#         'Class_II', 
#         'Class_I_II',
#         'Other_DESA', 
#         # 'No DESA',
#         'Relevant_DESA_Bad',
#         'Relevant_DESA_Good',
#     ], axis=1)

# cph.fit(df_new, weights_col='w', duration_col='T', event_col='E', robust=True)
# cph.print_summary()
cols = [
    'DonorAge_NOTR', 'RecipientAge_NOTR',
    'DialysisYears', 'IL2rMoAb',
    'CIPHour_DBD', 'CIPHour_DCD', 
    'No_DESA',
    'Specific_DESA',
    'Class_I', 
    'Class_II', 
    # 'Class_I_II',
    'E', 'T', 'w',
]
df_new = df_weight[cols]
cph.fit(df_new, weights_col='w', duration_col='T', event_col='E', robust=True)
cph.print_summary()
# df_weight[cols]


<lifelines.CoxPHFitter: fitted with 9687.9 total observations, 5824.78 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 9687.9
number of events observed = 3863.13
   partial log-likelihood = -33010.96
         time fit was run = 2022-02-11 09:39:25 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
DonorAge_NOTR       1.00       2.72       0.40             0.22             1.78                 1.25                 5.91
RecipientAge_NOTR  -1.12       0.33       0.40            -1.91            -0.33                 0.15                 0.72
DialysisYears       0.04       1.04       0.05            -0.06             0.13                 0.94                 1.14
IL2rMoAb            0.01       1.01       0.20            -0.37             0.40                 0.69                 1.49
CIPHour_DBD         0.65       1.93       0.52            -0.36             1.67                 0.70                 5.30
CIPHour_DCD         1.41       4.10       0.47             0.50             2.33                 1.64                10.23
No_DESA            -1.07       0.34       0.35            -1.75            -0.39                 0.17                 0.68
Specific_DESA       0.99       2.70       0.17             0.66             1.33                 1.94                 3.77
Class_I            -0.87       0.42       0.32            -1.50            -0.24                 0.22                 0.79
Class_II           -0.66       0.52       0.32            -1.29            -0.03                 0.28                 0.97

                      z      p   -log2(p)
covariate                                
DonorAge_NOTR      2.52   0.01       6.41
RecipientAge_NOTR -2.77   0.01       7.47
DialysisYears      0.73   0.46       1.10
IL2rMoAb           0.06   0.95       0.07
CIPHour_DBD        1.27   0.21       2.28
CIPHour_DCD        3.03 <0.005       8.66
No_DESA           -3.06 <0.005       8.84
Specific_DESA      5.84 <0.005      27.52
Class_I           -2.69   0.01       7.12
Class_II          -2.04   0.04       4.59
---
Concordance = 0.65
Partial AIC = 66041.92
log-likelihood ratio test = 1701.13 on 10 df
-log2(p) of ll-ratio test = inf

In [54]:
cph.check_assumptions(df_new, p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:307: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  for variable in self.params_.index & (colum

[]

In [53]:
from lifelines.statistics import proportional_hazard_test

results = proportional_hazard_test(cph, df_new, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9687.9 total observations, 5824.78 right-censored observations>
         test_name = proportional_hazard_test

---
                   test_statistic    p  -log2(p)
CIPHour_DBD                  0.11 0.74      0.44
CIPHour_DCD                  0.12 0.73      0.46
Class_I                      0.01 0.92      0.12
Class_II                     0.00 0.95      0.08
DialysisYears                0.03 0.86      0.22
DonorAge_NOTR                0.00 1.00      0.01
IL2rMoAb                     0.00 0.95      0.07
No_DESA                      0.03 0.87      0.20
RecipientAge_NOTR            0.29 0.59      0.77
Specific_DESA                0.02 0.90      0.16

# Relevant DESA Good

In [55]:
df_new = df.copy(deep=True)

# poly_col = ['RecipientAge_NOTR', 'DonorAge_NOTR']
# for col in poly_col:
#     df_new[col +'^2'] = df_new[col] * df_new[col]
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df_new, RELEVANT_DESA_GOOD)
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
# treatments = ['Relevant_DESA_Bad', 'Relevant_DESA_Good']
df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)

cph = CoxPHFitter()
cols = [
    'DonorAge_NOTR', 'RecipientAge_NOTR',
    'DialysisYears', 'IL2rMoAb',
    'CIPHour_DBD', 'CIPHour_DCD', 
    'No_DESA',
    'Specific_DESA',
    'Class_I', 
    'Class_II', 
    # 'Class_I_II',
    'E', 'T', 'w',
]
df_new = df_weight[cols]

cph.fit(df_new, weights_col='w', duration_col='T', event_col='E', robust=True)

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 9688.9 total observations, 6110.86 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 9688.9
number of events observed = 3578.04
   partial log-likelihood = -30694.93
         time fit was run = 2022-02-11 09:45:39 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
DonorAge_NOTR       1.25       3.50       0.37             0.53             1.98                 1.69                 7.24
RecipientAge_NOTR  -1.21       0.30       0.39            -1.96            -0.45                 0.14                 0.64
DialysisYears       0.07       1.07       0.05            -0.03             0.17                 0.97                 1.19
IL2rMoAb           -0.08       0.92       0.17            -0.42             0.25                 0.66                 1.29
CIPHour_DBD         0.85       2.35       0.50            -0.13             1.84                 0.88                 6.31
CIPHour_DCD         1.64       5.18       0.43             0.80             2.48                 2.23                11.99
No_DESA            -1.46       0.23       0.36            -2.16            -0.75                 0.12                 0.47
Specific_DESA      -0.73       0.48       0.21            -1.14            -0.32                 0.32                 0.73
Class_I            -0.46       0.63       0.35            -1.15             0.22                 0.32                 1.25
Class_II           -0.44       0.64       0.34            -1.10             0.22                 0.33                 1.24

                      z      p   -log2(p)
covariate                                
DonorAge_NOTR      3.38 <0.005      10.45
RecipientAge_NOTR -3.13 <0.005       9.18
DialysisYears      1.32   0.19       2.42
IL2rMoAb          -0.48   0.63       0.66
CIPHour_DBD        1.70   0.09       3.48
CIPHour_DCD        3.84 <0.005      12.96
No_DESA           -4.07 <0.005      14.35
Specific_DESA     -3.49 <0.005      11.01
Class_I           -1.32   0.19       2.43
Class_II          -1.32   0.19       2.42
---
Concordance = 0.64
Partial AIC = 61409.86
log-likelihood ratio test = 1340.28 on 10 df
-log2(p) of ll-ratio test = 933.83

In [56]:
cph.check_assumptions(df_new, p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:307: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  for variable in self.params_.index & (colum

[]

In [57]:
from lifelines.statistics import proportional_hazard_test

results = proportional_hazard_test(cph, df_new, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9688.9 total observations, 6110.86 right-censored observations>
         test_name = proportional_hazard_test

---
                   test_statistic    p  -log2(p)
CIPHour_DBD                  0.09 0.76      0.39
CIPHour_DCD                  0.11 0.74      0.43
Class_I                      0.01 0.94      0.08
Class_II                     0.00 0.97      0.04
DialysisYears                0.03 0.87      0.20
DonorAge_NOTR                0.00 0.97      0.04
IL2rMoAb                     0.00 0.97      0.05
No_DESA                      0.02 0.89      0.16
RecipientAge_NOTR            0.27 0.60      0.73
Specific_DESA                0.00 0.96      0.06